# Implementation of FORCE learning

1. Overview
2. Purpose
3. Preparation
4. Implementation
    - Closed-loop setup
    - FORCE module (RLS algorithm)
    - Online training control
5. Demonstrations

## 1. Overview

Here, let's implement the FORCE learning method proposed by [D. Sussillo *et al.,* (2009)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2756108/).

FORCE learning is an online training method for training and controlling linear closed-loop by capitalizing on an adaptive filter algorithm called [recursive least square (RLS) filter](https://en.wikipedia.org/wiki/Recursive_least_squares_filter).
Though an online method with an RLS filter has already been proposed and demonstrated in H. Jaeger *et al.,* (2004), the main difference between FORCE learning and the other RC methods lies in the network condition, that is, FORCE learning utilizes a **chaotic ESN** as the initial setup.
It would sound weird for those familiar with RC since it does not guarantee the echo state property, but FORCE learning nicely utilizes
 and 'harnesses' the system's chaoticity to realize the desired dynamics.

Let's implement FORCE learning and intuitively understand the mechanism in this notebook.

## 2. Purpose
Implementing an online training setup with FORCE learning and replicating the paper's results predicting chaotic time series data.

## 3. Preparation
Just run the following importing scripts (press `shift+Enter` to run each cell).
You will see an authentification popup, then please accept it.

In [ ]:
import sys
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
if "google.colab" in sys.modules:
    from google.colab import drive  # Connecting to google drive
    drive.mount("/content/gdrive")
    %cd /content/gdrive/My Drive/sandbox/rc_tutorial/closed_loop_control/
    # NOTE: change it to your own path
    # e.g., %cd /content/gdrive/My Drive/[PATH_TO_NOTEBOOK]
    from utils.magic import colab_script  # save and load cells
    exec(colab_script)
else:
    sys.path.append(".")
    from utils.magic import run_and_save, load_and_run  # save and load cells

from utils.tqdm import tqdm  # progress bar
from utils.reservoir import Module, ESN  # class for echo state network
from views.force_learning import *  # visualization wrapper
from tests.force_learning import *  # check function

## 4. Implementation
### 4.1 Closed loop setup
We used the following leaky echo state network (LESN) setups ([H. Jaeger *et al.*, 2007](https://pubmed.ncbi.nlm.nih.gov/17517495/));
$$
\def\bm#1{{\boldsymbol #1}}
\begin{align*}\\
\bm{x}(t+1) &= (1-a)\bm{x}(t)+a\tanh\left(g W^\text{net} \bm{x}(t) + W^\text{feed}\bm{y}(t)\right)\\
\bm{y}(t) &= W^{\text{out}} \bm{x}(t),
\\\end{align*}$$
where $a$ is the leaky rate (we will use $a=0.1$), $W^{\text{out}} \in \mathbb{R}^{D \times N}$  a linear readout outputting $D$-dimensional vector, and $W^\text{feed} \in \mathbb{R}^{N \times D}$ a linear feedback loop feeding it back to the system.

Please check `ESN.step` and how it is implemented at `./utils/reservoir.py`.

### 4.2 FORCE module (RLS algorithm)
Next, let's implement FORCE learning.

FORCE learning is an online training method; *i. e.*, the readout weight is updated at every time step.
In particular, it has demonstrated its outstanding effectiveness by incorporating the recursive least square (RLS) algorithm (Note that FORCE does **NOT** necessarily need to use the RLS algorithm. Please check the alternative algorithm in the supplementary material in the original paper). 
The weight update process of the RLS algorithm can be written in the following equation;
$$
\def\bm#1{{\boldsymbol #1}}
\begin{align*}\\
\bm{k} &= P\bm{x} \\
g &= \frac{1}{1+\bm{x}^T\bm{k}} \\
\Delta\bm{P} &= g\bm{k}\bm{k}^{T} \\ 
\Delta W^{\text{out}} &= g (d - y) \bm{k} \\
P &\leftarrow P - \Delta P \\
W &\leftarrow W + \Delta W, \\
\\\end{align*}
$$
where $P \in \mathbb{R}^{N\times N}$ is an auxiliary matrix (positive definite), $y$ the system output, $d$ the desired output, and $\bm{x}$ the system state. The RLS algorithm is equivalent to Ridge regression with the regularization term $\alpha$ by setting the initial value of $P(0)$ to $I/\alpha$, i.e.;
$$\begin{align*}\\
P(T)&=\left(\sum_{t=1}^{T} \bm{x}(t)\bm{x}^{T}(t) + \alpha I\right)^{-1} \\
&=\left(X^T X + \alpha I\right)^{-1},
\\\end{align*}$$
where $X=[\bm{x}^T(1); \bm{x}^T(2); \cdots \bm{x}^T(T)]\in \mathbb{R}^{T\times N}$.
In other words, the RLS algorithm efficiently calculates the matrix inverse iteratively by capitalizing on an equation called [Woodbury matrix identity](https://en.wikipedia.org/wiki/Woodbury_matrix_identity);
$$\begin{align*}\\
(A+BDC)^{-1} = A^{-1} - A^{-1}B(D^{-1}+CA^{-1}B)^{-1}CA^{-1}.
\\\end{align*}$$
Please check the correctness of the update process by assigning $A=P^{-1}, B=\bm{x}, C=\bm{x}^T, D=1$.

Let's implement the `ForceReadout` by overriding `Linear`.
Fill in the following blank in the `rls_update` implementing the RLS update.

Specification of `rls_update`
- Arguments
    - `P` (`np.ndarray`): Auxiliary matrix $P$ ($\in \mathbb{R}^{N\times N}$).
    - `x` (`np.ndarray`): Input state $\bm{x}$ ($\in \mathbb{R}^{N}$).
- Return
    - `np.ndarray`: Updated matrix $P-\Delta P$ ($\in \mathbb{R}^{N \times N}$) for $\mathcal{L}^\text{Ridge}$.

Specification of `ForceReadout.step`
- Arguments
    - `x` (`np.ndarray`): Input state $\bm{x}$ ($\in \mathbb{R}^{\cdots \times N}$).
    - `d` (`np.ndarray`): Desired output $\bm{x}$ ($\in \mathbb{R}^{\cdots \times D}$).
- Return
    - `np.ndarray`: Difference of matrix ($\Delta W$)
- Operation
    - Update `ForceReadout.P` with `rls_update`.
    - Update `ForceReadout.w` with the obtained weight.

<details>
<summary>
Hints
</summary>

- Use `np.dot`
- Use `np.outer`
- Arguments of `step`
  - $\bm{x}$: system state $\bm{x} \in \mathbb{N}$
  - $\bm{d}$: desired outputs $\bm{d} \in \mathbb{D}$

In [ ]:
%%run_and_save ./answer/rls_update.py

def rls_update(P, x):
    k = ... # TODO
    g = ... # TODO
    dP = ... # TODO
    P_new = ... # TODO
    return g, k, P_new

In [ ]:
test_rls_update(rls_update)

In [ ]:
%load_and_run ./answer/linear.py

class ForceReadout(Linear):
    def __init__(self, *args, alpha=1.0, **kwargs):
        super(ForceReadout, self).__init__(*args, **kwargs)
        self.alpha = alpha
        self.Ps = []
        for _ in range(self.output_dim):
            P = ... # TODO create initial value of P
            self.Ps.append(P)

    def step(self, x, d):
        assert x.ndim == 1
        e = d - self(x)
        dws = np.zeros_like(self.w)
        for idx in range(self.output_dim):
            P = self.Ps[idx]
            g, k, P_new = ... # TODO
            dw = ... # TODO
            self.Ps[idx] = P_new
            self.w[idx, :] += dw
            dws[idx, :] = dw
        return dws

In [ ]:
test_force_readout(ForceReadout)

### 4.3 Online training control

Next, let's implement a function controlling the training and evaluation phase. 

Please fill the blank in `emulate`.

Specification of `emulate`:
- Arguments:
    - `ts` (`list` or `np.ndarray` or `range`): List of sampled time (*e. g.*, $[0, 1, 2, ..., T-1]$).
    - `net` (`ESN`): An instance for DESN ($\tanh, W^\text{net}$).
    - `w_feed` (`Linear` or `RidgeReadout`): An instance of $W^\text{feed}$.
    - `w_out` (`Linear` or `RidgeReadout`): An instance of $W^\text{out}$.
    - `ds` (`list` or `np.ndarray`): Desired time series data $\bm{d}(t)$.
    - `force_every` (`int`): Frequency of `ForceReadout.train` call.
    - `train_range` (`tuple(int, int)`): Time range of training (*i.e.* training phase, $[t_0, t_1)$).
- Return
    - `dict`: All records.

<details>
<summary>
Hints
</summary>

- Call `w_out.step`
- Use `net.x`

In [ ]:
def emulate(ts, net, w_feed, w_out, ds=None, force_every=1, train_range=[0, 0]):
    T = len(ts)
    record = {}
    record["train_range"] = train_range
    record["t"] = np.zeros(T)
    record["x"] = np.zeros((T, *net.x.shape))
    record["y"] = np.zeros((T, *w_out(net.x).shape))
    record["d"] = np.zeros((T, *w_out(net.x).shape))
    record["W"] = np.zeros((T + 1, *w_out.w.shape))
    record["W"][0] = w_out.w
    pbar = tqdm(ts)
    for idx, t in enumerate(pbar):
        y = w_out(net.x)
        d = ds[idx] if (ds is not None) and (idx < len(ds)) else 0.0
        if idx % force_every == 0 and (train_range[0] <= t < train_range[1]):
            dws = ... # TODO update w_out
            pbar.set_description("|ΔW|={:.3e}".format(np.linalg.norm(dws)))
        record["t"][idx] = t
        record["x"][idx] = net.x
        record["y"][idx] = y
        record["d"][idx] = d
        record["W"][idx + 1] = w_out.w
        net.step(w_feed(y))
    return record

## 5 Demonstration
Now, it's time to use FORCE learning.

### 5.1 Periodic function
First, let's embed the periodic function shown in the original paper.

Use the following sinusoid function.

$$
\def\bm#1{{\boldsymbol #1}}
\begin{align*}\\
d(t) = 
\frac{2}{3} \sin\left(\frac{\pi t}{T}\right) + 
\frac{1}{3} \sin\left(\frac{2\pi t}{T}\right) + 
\frac{1}{9} \sin\left(\frac{3\pi t}{T}\right) + 
\frac{2}{9} \sin\left(\frac{4\pi t}{T}\right),
\\\end{align*}
$$
where the time constant $T=60$.


In [ ]:
ts = np.linspace(0, 20000, 20001)

amps = [2/3, 1/3, 1/9, 2/9]
freqs = [1.0, 2.0, 3.0, 4.0]

ys_p = np.zeros_like(ts)
for a, f in zip(amps, freqs):
    ys_p += a * np.sin(f * np.pi * ts / 60)
ys_p = ys_p / 1.5

fig = show_trajectory(ys_p[:1000])

Prepare 1000-dimensional LESN setup with leaky rate 0.1.

Since the target signal is one-dimensional, `out_dim` is set to 1.

In [ ]:
seed = 1234
rnd = np.random.RandomState(seed)

net_dim = 1000
out_dim = 1

bias = rnd.uniform(-0.2, 0.2, net_dim)
net = ESN(net_dim, g=1.5, p=0.01, bias=bias, a=0.1, rnd=rnd)  # NOTE g is larger than 1, i.e., chaotic
w_feed = Linear(out_dim, net_dim, bound=5.0, rnd=rnd)
w_out = ForceReadout(net_dim, out_dim, bound=0.1, alpha=0.1, rnd=rnd)


Let's train the network.

In [ ]:
ts = range(5000)
train_range = [1000, 4000]
force_every = 2

record = emulate(ts, net, w_feed, w_out, ys_p,
                 force_every=force_every, train_range=train_range)
fig = show_record(record)

Also, let's check how successfully the system outputs the desired trajectory.

In [ ]:
record_eval = emulate(range(1000), net, w_feed, w_out)
fig = show_delayed_coord(record_eval["y"][:, 0], record["d"][:, 0], labels=["output", "desired"])


### 5.2 Lorenz attractor
Next, let's try chaotic time-series prediction.

In [ ]:
%load_and_run ./answer/odeint.py
%load_and_run ./answer/lorenz_system.py

ts = np.linspace(0, 200, 20001)
ys_lz = odeint(lorenz_eqn, [1.0, 1.0, 1.0], ts)

fig = show_3d_coord(lorenz=ys_lz[:5000])

In [ ]:
fig.show()

Note that `out_dim` is set to 3.

In [ ]:
out_dim = 3

w_feed = Linear(out_dim, net_dim, bound=5.0, rnd=rnd)
w_out = ForceReadout(net_dim, out_dim, bound=0.0, alpha=0.1, rnd=rnd)

Let's train the network.

In [ ]:
ts = range(5000)
train_range = [1000, 4000]
force_every = 2

net.reset(np.random.uniform(-1, 1, net.dim))

record = emulate(ts, net, w_feed, w_out, ys_lz * 0.01,
                 force_every=force_every, train_range=train_range)
fig = show_record(record)

Now, check the reconstructed attractor.

Can you see the double-wing shape as expected?

In [ ]:
record_eval = emulate(range(10000), net, w_feed, w_out, ds=ys_lz[len(ts):] * 0.01)
fig = show_3d_coord(output=record_eval["y"], desired=record_eval["d"])

In [ ]:
fig.show()

### 5.3 Multiple attractors

You can also embed multiple attractors within the same system.
$$
\def\bm#1{{\boldsymbol #1}}
\begin{align*}\\
\bm{x}(t+1) &= (1-a)\bm{x}(t)+a\tanh\left(g W^\text{net} \bm{x}(t) + W^\text{feed}\bm{y}(t) + W^\text{in} u(t)\right)\\
\bm{y}(t) &= W^{\text{out}} \bm{x}(t),
\\\end{align*}$$
where $u$ is a control parameter $W^\text{in} \in \mathbb{R}^{N}$ is a matrix to project $u$ into the high-dimensional system. Let's design a feedback loop to output the Lorenz attractor and another butterfly-shaped attractor proposed by [L. Choingxin *et al.,* (2006)](https://www.sciencedirect.com/science/article/pii/S0960077904005909) when setting $u=-1$ and $u=1$, respectively.

To realize it, implement `emulate_parallel` by updating the `emulate` function to support the parallel emulation.

Specification of `emulate_parallel`:
- Arguments:
    - `ts` (`list` or `np.ndarray`): List of sampled time (*e. g.*, $[0, 1, 2, ..., T-1]$).
    - `net` (`ESN`): An instance for DESN ($\tanh, W^\text{net}$).
    - `w_feed` (`Linear` or `RidgeReadout`): An instance of $W^\text{feed}$.
    - `w_out` (`Linear` or `RidgeReadout`): An instance of $W^\text{out}$.
    - `ds` (`list` or `np.ndarray`): Desired time series data $\bm{d}(t)$.
    - `force_every` (`int`): Frequency of `ForceReadout.train` call.
    - `train_range` (`tuple(int, int)`): Time range of training (*i.e.* training phase, $[t_0, t_1)$).
- Return
    - `dict`: All records.

<details>
<summary>
Hints
</summary>

- `net.x` and `d` can be a multi-dimensional tensor (*i.e.*, $x(t)\in \mathbb{R}^{\cdots \times N}, d \in \mathbb{R}^{\cdots \times D}$)
- However, the arguments $x$ for `w_out.step` must be a vector.

In [ ]:
def emulate_parallel(ts, net, w_feed, w_out, w_in, us=None, ds=None, force_every=1, train_range=[0, 0]):
    T = len(ts)
    record = {}
    record["train_range"] = train_range
    record["t"] = np.zeros(T)
    record["x"] = np.zeros((T, *net.x.shape))
    record["y"] = np.zeros((T, *w_out(net.x).shape))
    record["d"] = np.zeros((T, *w_out(net.x).shape))
    record["W"] = np.zeros((T + 1, *w_out.w.shape))
    record["W"][0] = w_out.w
    pbar = tqdm(ts)
    for idx, t in enumerate(pbar):
        y = w_out(net.x)
        d = ds[idx] if (ds is not None) and (idx < len(ds)) else 0.0
        u = us[idx] if (us is not None) and (idx < len(us)) else 0.0
        if idx % force_every == 0 and (train_range[0] <= t < train_range[1]):
            if net.x.ndim == 1:
                dws = ... # TODO
            elif net.x.ndim == 2:
                dws = []
                for idy in range(net.x.shape[0]):
                    out = ... # TODO
                    dws.append(out)
            pbar.set_description("|ΔW|={:.3e}".format(np.linalg.norm(dws)))
        record["t"][idx] = t
        record["x"][idx] = net.x
        record["y"][idx] = y
        record["d"][idx] = d
        record["W"][idx + 1] = w_out.w
        net.step(w_feed(y) + w_in(u))
    return record

Check the alternative attractor.

In [ ]:
def chongxin_eqn(x, t, a=10, b=40, k=1, c=2.5, h=4):
    x_dot = np.zeros_like(x)
    x_dot[0] = a * (x[1] - x[0])
    x_dot[1] = b * x[0] - k * x[0] * x[2]
    x_dot[2] = -c * x[2] + h * x[0] * x[0]
    return x_dot

ts = np.linspace(0, 200, 20001)
ys_cx = odeint(chongxin_eqn, [1.0, 1.0, 1.0], ts)
fig = show_3d_coord(lorenz=ys_lz[:5000], chongxin=ys_cx[:5000])

In [ ]:
fig.show()

Let's embed multiple attractors with a single readout.

In [ ]:
out_dim = 3

w_in = Linear(1, net_dim, bound=0.1, rnd=rnd)
w_feed = Linear(out_dim, net_dim, bound=5.0, rnd=rnd)
w_out = ForceReadout(net_dim, out_dim, bound=0.0, alpha=0.1, rnd=rnd)

In [ ]:
ts = range(10000)
train_range = [1000, 5000]
force_every = 2

net.reset(rnd.uniform(-1, 1, (2, net.dim)))
ys_all = np.array([ys_lz * 0.01, ys_cx * 0.01]).swapaxes(0, 1)
us_all = np.zeros((len(ys_all), 2, 1))
us_all[:, 1] = 1  # u=1 for lorenz
us_all[:, 0] = -1  # u=-1 for chongxin

record = emulate_parallel(
    ts, net, w_feed, w_out, w_in, us=us_all, ds=ys_all,
    force_every=force_every, train_range=train_range)
fig1 = show_record(record, index=0)  # lorenz
fig2 = show_record(record, index=1)  # chongxin

In [ ]:
fig1.show()

In [ ]:
fig2.show()

In [ ]:
record_eval = emulate_parallel(
    range(10000), net, w_feed, w_out, w_in, 
    us=us_all[len(ts):], ds=ys_all[len(ts):])
fig = show_3d_coord(
    lorenz_out=record_eval["y"][:, 0], lorenz_des=record_eval["d"][:, 0],
    chongxin_out=record_eval["y"][:, 1], chongxin_des=record_eval["d"][:, 1])

In [ ]:
fig.show()

## Advanced topics

- Read Sussillo's **Supporting Online Material** in the original paper
- Implement a version with **input signal**
- Try the non-chaotic condition
    - The paper claims that the performance worsens in a non-chaotic regime
- Quantitatively compare the online results with offline ones